In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import pandas_datareader.data as web
import math
import ta
import keras
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow.compat.v2.feature_column as fc
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout, BatchNormalization, Concatenate
from keras import optimizers
from keras.utils import plot_model
import matplotlib.pyplot as plt
from ta.utils import dropna
from six.moves import urllib
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


# Hidden Markov Model 

In [2]:
###dataframe of the stock
ticker = input('Ticker: ')
days = input('Days: ')
def dfStock(ticker,start,end):
    data = web.DataReader(ticker.upper(),'yahoo',start,end)[['Adj Close']]
    data['Daily Return'] = (data['Adj Close'].pct_change())*100
    data['target'] = int(False)
    data.loc[data['Daily Return'] > 0, 'target']=int(True)
    data['up'] = int(False)
    data.loc[data['target'] == 1, 'up']=int(True)
    data['down'] = int(False)
    data.loc[data['target'] == 0, 'down']=int(True)
    data = data.dropna() 
    data = data[['Adj Close','Daily Return','target','up','down']]
    return data
###initial probability of having a positive or negative return
df = dfStock(ticker,dt.datetime.now()-dt.timedelta(days=int(days)),dt.datetime.now())
daily_prob = df.groupby('target').size().div(len(df))
daily_prob

Ticker: evri
Days: 200


target
0    0.507353
1    0.492647
dtype: float64

In [3]:
###transition probability of having a previous positive or negative day    

from itertools import islice
def window(seq, n=2):
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result
    
pairs = pd.DataFrame(window(df['target']), columns=['state1', 'state2'])
counts = pairs.groupby('state1')['state2'].value_counts()
transition_prob_neg = (counts[0] / counts[0].sum())
transition_prob_pos = (counts[1] / counts[1].sum())

In [4]:
#mean and standard deviation of up and down days
avg = df.groupby('target')['Daily Return'].mean()
stdev = df.groupby('target')['Daily Return'].std()

In [5]:
tfd = tfp.distributions  
#probability of next day being down or up
initial_distribution = tfd.Categorical(probs=[daily_prob[0].astype('float32'), daily_prob[1].astype('float32')])  
#transition probabilities (one event leading to another) (i.e. if the stock is down what prob of it being the next day)
transition_distribution = tfd.Categorical(probs=[[transition_prob_neg[0].astype('float32'), transition_prob_neg[1].astype('float32')],
                                                 [transition_prob_pos[0].astype('float32'),transition_prob_pos[1].astype('float32')]])  
#mean and std (loc = mean & scale = standard deviation)
observation_distribution = tfd.Normal(loc=[avg[0].astype('float32'), avg[1].astype('float32')], scale=[stdev[0].astype('float32'), stdev[1].astype('float32')])  

In [6]:
#number of steps is how far we want to predict into the future
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution=observation_distribution,
    num_steps=5)

In [7]:
#prediction for the next 5 days daily returns
mean = model.mean()
with tf.compat.v1.Session() as sess:  
    print(mean.numpy())

[0.12176681 0.07500458 0.07398844 0.07396603 0.07396531]
